In [14]:
!pip install tensorflow-io


In [15]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import polars as pl
import tensorflow_io as tfio
from PIL import Image
import numpy as np
import io

load_dotenv()

True

In [16]:
def load_and_convert_image(filepath: str):
    full_path = os.path.join(os.environ["PATH_ARCHIVE"], filepath.replace("/", os.sep))
    im = Image.open(full_path)
    return np.array(im).tobytes()


def load_and_convert_image_2(filepath: str):
    full_path = os.path.join(os.environ["PATH_ARCHIVE"], filepath.replace("/", os.sep))
    im = Image.open(full_path)
    image_bytes = io.BytesIO()
    im.save(image_bytes, format="JPEG")
    return image_bytes.getvalue()

In [17]:
# setup database
client = MongoClient(os.environ["URI"])

# Get reference to 'bank' database
db = client.flowers

# Get a reference to 'magazines' collection
train_collection = db.train

train_df = pl.read_csv(os.path.join(os.environ["PATH_ARCHIVE"], "train.csv"))

In [18]:
result = train_collection.delete_many({})
result.deleted_count

0

In [19]:
# remove the paths from the df
train_df = train_df.with_columns(pl.col("image:FILE").str.split(by="/").alias("split"))
train_df = train_df.select(
    pl.col("split").list.get(2).str.strip_suffix("_c.jpg").alias("id"),
    pl.col("split").list.get(1).alias("flower_type"),
    pl.col("category"),
    pl.col("image:FILE").map_elements(load_and_convert_image_2, return_dtype=pl.Binary).alias("data"),
    pl.col("image:FILE").alias("filepath"),
)

In [20]:
# train_df.head()
filepath = train_df.row(0, named=True)["filepath"]
print(len(load_and_convert_image(filepath)))
print(len(load_and_convert_image_2(filepath)))
print(len(train_df.row(0, named=True)["data"]))

196608
12108
12108


In [21]:
# load_and_convert_image(train_df.row(0, named=True)["filepath"])
id_list = train_df.get_column("id").to_list()

In [22]:
result = train_collection.insert_many(
    train_df.to_dicts()
)

document_ids = result.inserted_ids
print("# of documents inserted: " + str(len(document_ids)))
print(f"_ids of inserted documents: {document_ids}")

# of documents inserted: 13642
_ids of inserted documents: [ObjectId('654e9ee42a4eb81aa00d5562'), ObjectId('654e9ee42a4eb81aa00d5563'), ObjectId('654e9ee42a4eb81aa00d5564'), ObjectId('654e9ee42a4eb81aa00d5565'), ObjectId('654e9ee42a4eb81aa00d5566'), ObjectId('654e9ee42a4eb81aa00d5567'), ObjectId('654e9ee42a4eb81aa00d5568'), ObjectId('654e9ee42a4eb81aa00d5569'), ObjectId('654e9ee42a4eb81aa00d556a'), ObjectId('654e9ee42a4eb81aa00d556b'), ObjectId('654e9ee42a4eb81aa00d556c'), ObjectId('654e9ee42a4eb81aa00d556d'), ObjectId('654e9ee42a4eb81aa00d556e'), ObjectId('654e9ee42a4eb81aa00d556f'), ObjectId('654e9ee42a4eb81aa00d5570'), ObjectId('654e9ee42a4eb81aa00d5571'), ObjectId('654e9ee42a4eb81aa00d5572'), ObjectId('654e9ee42a4eb81aa00d5573'), ObjectId('654e9ee42a4eb81aa00d5574'), ObjectId('654e9ee42a4eb81aa00d5575'), ObjectId('654e9ee42a4eb81aa00d5576'), ObjectId('654e9ee42a4eb81aa00d5577'), ObjectId('654e9ee42a4eb81aa00d5578'), ObjectId('654e9ee42a4eb81aa00d5579'), ObjectId('654e9ee42a4eb81aa0

In [23]:
client.close()